In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df =pd.read_excel("Online_Retail.xlsx")
df.head()
df1 = df

In [3]:
pwd()

'C:\\Users\\Ghost\\Data_Science_Practise'

In [5]:
df1.Country.nunique()

38

In [6]:
df1.Country.nunique()

38

In [47]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID',ascending=False)



,Country,CustomerID
0,United Kingdom,3921


In [16]:
df1 = df1.loc[df1['Country'] == 'United Kingdom']

In [17]:
df1 = df1[pd.notnull(df1['CustomerID'])]

In [18]:
df1 = df1[pd.notnull(df1['CustomerID'])]

In [8]:
df1.Quantity.min()?

-80995

In [19]:
df1 = df1[pd.notnull(df1['CustomerID'])]

In [20]:
df1 = df1[pd.notnull(df1['CustomerID'])]

In [21]:
df1.Quantity.min()

-80995

In [22]:
df1 = df1[(df1['Quantity']>0)]
df1.shape
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 8 columns):
InvoiceNo      354345 non-null object
StockCode      354345 non-null object
Description    354345 non-null object
Quantity       354345 non-null int64
InvoiceDate    354345 non-null datetime64[ns]
UnitPrice      354345 non-null float64
CustomerID     354345 non-null float64
Country        354345 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


In [25]:
def unique_counts(df1):
    for i in df1.columns:
        count = df1[i].nunique()
        print(i, ": ", count)
unique_counts(df1)

InvoiceNo :  16649
StockCode :  3645
Description :  3844
Quantity :  294
InvoiceDate :  15615
UnitPrice :  403
CustomerID :  3921
Country :  1


In [26]:
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']

In [27]:
df1['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [28]:
df1['InvoiceDate'].max()

Timestamp('2011-12-09 12:49:00')

In [31]:
import datetime as dt
NOW = dt.datetime(2011,12,10)
df1['InvoiceDate'] =pd.to_datetime(df1['InvoiceDate'])

In [33]:
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x:(NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice':
lambda x: x.sum()})
rfmTable['InvoiceDate'] =rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency',
'InvoiceNo': 'frequency',

'TotalPrice': 'monetary_value'},

inplace=True)

In [35]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346.0,325,1,77183.60
12747.0,2,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,3,59,942.34


In [36]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [37]:
segmented_rfm = rfmTable

In [43]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4
    def FMScore(x,p,d):
        if x <= d[p][0.25]:
            return 4
        elif x <= d[p][0.50]:
            return 3
        elif x <= d[p][0.75]:
            return 2
        else:
            return 1
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1

In [44]:
segmented_rfm['r_quartile'] =segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] =segmented_rfm['frequency'].apply(FMScore,args=('frequency',quantiles,))
segmented_rfm['m_quartile'] =segmented_rfm['monetary_value'].apply(FMScore,args=('monetary_value',quantiles,))
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12346.0,325,1,77183.60,4,4,1
12747.0,2,103,4196.01,1,1,1
12748.0,0,4596,33719.73,1,1,1
12749.0,3,199,4090.88,1,1,1
12820.0,3,59,942.34,1,2,2


In [45]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str)+ segmented_rfm.f_quartile.map(str)+ segmented_rfm.m_quartile.map(str)

segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,4,4,1,441
12747.0,2,103,4196.01,1,1,1,111
12748.0,0,4596,33719.73,1,1,1,111
12749.0,3,199,4090.88,1,1,1,111
12820.0,3,59,942.34,1,2,2,122


In [46]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
15311.0,0,2379,60767.90,1,1,1,111
13089.0,2,1818,58825.83,1,1,1,111
15769.0,7,130,56252.72,1,1,1,111
